# Find SGLs Notebook Managed

The whole point of this notebook is that there are very few colocalization so why am I bothering with making scripts
that are run through qsub??

In [1]:
import os 
import glob
import sys
import pybedtools as pbt
import pandas as pd
import numpy as np
import subprocess as sp
import json
import argparse
from itertools import chain
pbt.helpers.set_tempdir('/mnt/BioHome/jreyna/tmp/')
pbt.set_bedtools_path('/mnt/BioApps/bedtools/bin/')
bgzip = '/mnt/BioApps/tabix/tabix-0.2.6/bgzip'
tabix = '/mnt/BioApps/tabix/tabix-0.2.6/tabix'

pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_colwidth', 400)

In [2]:
os.chdir('/mnt/BioHome/jreyna/jreyna/projects/dchallenge/')

In [3]:
# setting basical column names for bedpe
bedpe_6cols = ['chrA', 'startA', 'endA', 'chrB', 'startB', 'endB']
bedpe_10cols = ['chrA', 'startA', 'endA', 'chrB', 'startB', 'endB', 'name', 'score', 'strandA', 'strandB']

In [29]:
# pathing a few reference files
gs = 'results/refs/hg19/hg19.chrom.sizes'
gene_ref = 'results/refs/gencode/v30/gencode.v30.annotation.grch37.genes_only.bed'
outdir = 'results/main/GRCh37/sgls/notebook_output/Find_SGLs.Notebook_Managed.LD_SNPs_NOT_Included/'
os.makedirs(outdir, exist_ok=True)

## Loading the Gencode Information

In [30]:
gencode = pd.read_table(gene_ref, header=None)

In [31]:
gencode.columns = ['chr', 'gene_start', 'gene_end', 'gene_strand', 'type', 'geneid', 'genename']

## Loading the COLOC data

In [32]:
#eqtls = 'results/main/GRCh37/sgls/ImmuNexUT/T1D_34012112_Gaulton/ImmuNexUT/DN_B/eqtls.coloc_filtered.tsv.gz'
colocs = 'results/main/GRCh37/coloc/Results/*/*/*/*/FINAL_Summary_Coloc_Gene_SNP_Pairs.bed'
colocs = glob.glob(colocs)

In [33]:
len(colocs)

166

In [34]:
data = []
for fn in colocs:
    
    # get data source
    file_info = fn.split('/')  
    gwas_source = file_info[6]
    eqtl_source = file_info[7]
    ge_source = file_info[8]
    
    # load the data
    df = pd.read_table(fn)
    df['gwas_source'] = gwas_source
    df['eqtl_source'] = eqtl_source
    df['ge_source'] = ge_source
    data.append(df)
colocs_df = pd.concat(data)

In [35]:
colocs_df.shape

(351, 29)

In [36]:
# add snp_start and snp_end
colocs_df.loc[:, 'snp_start'] = colocs_df['pos'] - 1
colocs_df.loc[:, 'snp_end'] = colocs_df['pos']

In [37]:
colocs_df.head()

,chr,pos,pp_H0_Coloc_Summary,pp_H1_Coloc_Summary,pp_H2_Coloc_Summary,pp_H3_Coloc_Summary,pp_H4_Coloc_Summary,rs_id,variant_id,geneName,dist,pvalue,FDR,slope_snp,ref,alt,AC,AF,AN,slope_se_snp,slope_gwas,slope_se_gwas,pval_nominal,SampleSize,old_pos,was_converted,gwas_source,eqtl_source,ge_source,snp_start,snp_end
0,chr4,166575439,4.500129e-43,5.525419e-40,5.987878e-05,0.072594,0.927346,rs2611211,4:165654287:C:T,ENSG00000183439,676619,2.541920e-02,4.368856e-01,-0.267438,C,T,3735,0.745807,5008,0.018893,-0.143854,0.018689,1.390000e-14,520580.0,165654287,1,T1D_34012112_Gaulton,BLUEPRINT,T-cell,166575438,166575439
1,chr18,67533961,0.000000e+00,0.000000e+00,5.685054e-05,0.132827,0.867116,rs1788229,18:69866725:T:G,ENSG00000266840,141750,3.615440e-11,1.318455e-08,-0.564373,T,G,2540,0.507188,5008,0.002109,-0.102254,0.014003,2.830000e-13,520580.0,69866725,1,T1D_34012112_Gaulton,BLUEPRINT,T-cell,67533960,67533961
0,chr9,4290823,0.000000e+00,0.000000e+00,1.086753e-03,0.189642,0.809271,rs6476839,9:4290823:A:T,ENSG00000107249,57569,1.026900e-02,2.420917e-01,-0.338403,A,T,2438,0.486821,5008,0.006585,0.123270,0.014153,3.040000e-18,520580.0,4290823,1,T1D_34012112_Gaulton,BLUEPRINT,neutrophil,4290822,4290823
1,chr16,75252327,5.981813e-220,3.632565e-214,3.397276e-07,0.205511,0.794488,rs8056814,16:75218429:G:A,ENSG00000196436,840551,1.371830e-01,7.143720e-01,0.409247,G,A,906,0.180911,5008,0.012900,0.217762,0.024263,2.830000e-19,520580.0,75218429,1,T1D_34012112_Gaulton,BLUEPRINT,neutrophil,75252326,75252327
0,chr8,120083701,6.662446e-83,2.825264e-82,1.849050e-02,0.077506,0.904003,rs1485294,8:119071462:G:C,ENSG00000164761,119577,1.869290e-02,5.172063e-01,-0.262816,G,C,3209,0.640775,5008,0.013207,-0.091563,0.014668,4.310000e-10,NaN,119071462,1,T1D_34012112_Gaulton,Quach_2016,monocyte_R848,120083700,120083701


In [38]:
# add gene meta data
colocs_df = colocs_df.merge(gencode.drop(['chr', 'type'], axis=1),
                            left_on='geneName', right_on='geneid')

In [39]:
colocs_df

,chr,pos,pp_H0_Coloc_Summary,pp_H1_Coloc_Summary,pp_H2_Coloc_Summary,pp_H3_Coloc_Summary,pp_H4_Coloc_Summary,rs_id,variant_id,geneName,dist,pvalue,FDR,slope_snp,ref,alt,AC,AF,AN,slope_se_snp,slope_gwas,slope_se_gwas,pval_nominal,SampleSize,old_pos,was_converted,gwas_source,eqtl_source,ge_source,snp_start,snp_end,gene_start,gene_end,gene_strand,geneid,genename
0,chr4,166575439,4.500129e-43,5.525419e-40,5.987878e-05,0.072594,0.927346,rs2611211,4:165654287:C:T,ENSG00000183439,676619,2.541920e-02,4.368856e-01,-0.267438,C,T,3735,0.745807,5008,0.018893,-0.143854,0.018689,1.390000e-14,520580.0,165654287,1,T1D_34012112_Gaulton,BLUEPRINT,T-cell,166575438,166575439,165875598,165898820,-,ENSG00000183439,TRIM61
1,chr18,67533961,0.000000e+00,0.000000e+00,5.685054e-05,0.132827,0.867116,rs1788229,18:69866725:T:G,ENSG00000266840,141750,3.615440e-11,1.318455e-08,-0.564373,T,G,2540,0.507188,5008,0.002109,-0.102254,0.014003,2.830000e-13,520580.0,69866725,1,T1D_34012112_Gaulton,BLUEPRINT,T-cell,67533960,67533961,67372214,67392211,-,ENSG00000266840,AC068254.1
2,chr18,67532292,0.000000e+00,0.000000e+00,4.999652e-05,0.157526,0.842424,rs1788105,18:69865056:A:G,ENSG00000266840,140081,5.771950e-11,1.155408e-07,-0.885223,A,G,2355,0.470248,5008,0.000626,-0.102549,0.013996,2.350000e-13,520580.0,69865056,1,T1D_34012112_Gaulton,Schmiedel_2018,Th1-17_memory,67532291,67532292,67372214,67392211,-,ENSG00000266840,AC068254.1
3,chr18,67522821,0.000000e+00,0.000000e+00,4.922234e-02,0.120658,0.830120,rs34875178,18:69855585:C:T,ENSG00000266840,130610,9.556970e-04,1.486691e-01,0.574268,C,T,2322,0.463658,5008,0.002046,0.120003,0.018000,2.616780e-11,NaN,69855585,1,T1D_25751624,Schmiedel_2018,Tfh_memory,67522820,67522821,67372214,67392211,-,ENSG00000266840,AC068254.1
4,chr18,67543688,0.000000e+00,0.000000e+00,5.761961e-02,0.123618,0.818762,rs1788097,18:69876452:C:T,ENSG00000266840,151477,3.326780e-07,3.059556e-04,0.803605,C,T,2762,0.551518,5008,0.000860,0.117961,0.017994,5.543510e-11,NaN,69876452,1,T1D_25751624,Schmiedel_2018,Th1_memory,67543687,67543688,67372214,67392211,-,ENSG00000266840,AC068254.1
5,chr18,67531642,0.000000e+00,0.000000e+00,6.184876e-02,0.132765,0.805386,rs763361,18:69864406:T:C,ENSG00000266840,139431,4.194280e-06,1.935681e-03,-0.577081,T,C,2351,0.469449,5008,0.002016,0.116983,0.017997,8.032000e-11,NaN,69864406,1,T1D_25751624,Schmiedel_2018,Th17_memory,67531641,67531642,67372214,67392211,-,ENSG00000266840,AC068254.1
6,chr18,67525286,0.000000e+00,0.000000e+00,4.922234e-02,0.120658,0.830120,rs1790961,18:69858050:T:G,ENSG00000266840,133075,1.708730e-04,5.469749e-02,-0.462213,T,G,2345,0.468251,5008,0.003368,0.120003,0.018000,2.616780e-11,NaN,69858050,1,T1D_25751624,Schmiedel_2018,NK-cell_naive,67525285,67525286,67372214,67392211,-,ENSG00000266840,AC068254.1
7,chr18,67531642,0.000000e+00,0.000000e+00,6.069069e-02,0.149003,0.790306,rs763361,18:69864406:T:C,ENSG00000266840,139431,5.771950e-11,1.155408e-07,-0.885223,T,C,2351,0.469449,5008,0.000626,0.116983,0.017997,8.032000e-11,NaN,69864406,1,T1D_25751624,Schmiedel_2018,Th1-17_memory,67531641,67531642,67372214,67392211,-,ENSG00000266840,AC068254.1
8,chr9,4290823,0.000000e+00,0.000000e+00,1.086753e-03,0.189642,0.809271,rs6476839,9:4290823:A:T,ENSG00000107249,57569,1.026900e-02,2.420917e-01,-0.338403,A,T,2438,0.486821,5008,0.006585,0.123270,0.014153,3.040000e-18,520580.0,4290823,1,T1D_34012112_Gaulton,BLUEPRINT,neutrophil,4290822,4290823,3824127,4348392,-,ENSG00000107249,GLIS3
9,chr9,4282536,0.000000e+00,0.000000e+00,9.161704e-04,0.192281,0.806803,rs4380994,9:4282536:A:G,ENSG00000107249,65856,6.439830e-03,3.367867e-01,-0.407677,A,G,1754,0.350240,5008,0.005361,-0.121582,0.013995,3.710000e-18,NaN,4282536,1,T1D_34012112_Gaulton,Quach_2016,monocyte_R848,4282535,4282536,3824127,4348392,-,ENSG00000107249,GLIS3


In [40]:
# add a coloc idd
def get_coloc_id(sr):
    s = '{}.{}.{}.{}.{}.{}'.format(sr.chr, sr.pos, sr.genename, sr.gwas_source, sr.eqtl_source, sr.ge_source)
    return(s)
colocs_df.loc[:, 'coloc_id'] = colocs_df.apply(get_coloc_id, axis=1)

# create a pybedtools object
colocs_gene_bedpe = colocs_df.loc[:, ['chr', 'snp_start', 'snp_end', 'chr', 'gene_start', 'gene_end', 'coloc_id']]
colocs_gene_pbt = pbt.BedTool.from_dataframe(colocs_gene_bedpe)

In [41]:
def get_tss_cords(sr):
    if sr.gene_strand == '+':
        return([sr.gene_start, sr.gene_start + 1])
    else:
        return([sr.gene_end - 1, sr.gene_end])
tss_starts, tss_ends = list(zip(*colocs_df.apply(get_tss_cords, axis=1)))
colocs_df['tss_start'] = tss_starts
colocs_df['tss_end'] = tss_ends

In [42]:
# create a pybedtools object
colocs_tss_bedpe = colocs_df.loc[:, ['chr', 'snp_start', 'snp_end', 'chr', 'tss_start', 'tss_end', 'coloc_id']]
colocs_tss_pbt = pbt.BedTool.from_dataframe(colocs_tss_bedpe)

## Intersecting COLOCs with Loops

In [43]:
loops = 'results/main/h3k27ac_hichip/*/FitHiChIP_S/FitHiChIP.interactions_FitHiC_Q0.01.bed'
loops = sorted(glob.glob(loops))
loops = loops[0:3] + [loops[11]]

In [44]:
# add loop id
def get_loop_id(sr):
    s = '{}.{}.{}.{}.{}.{}'.format(sr.chr1, sr.s1, sr.e1, sr.chr2, sr.s2, sr.e2)
    return(s)

In [45]:
tss_slop = 5000
gene_slop = 0

In [46]:
compiled_sgls = []
for loop_fn in loops:
    
    # get file sources
    file_info = loop_fn.split('/')
    loop_source = file_info[3]
    
    # load the loop data
    loop_df = pd.read_table(loop_fn)
    loop_df['loop_id'] = loop_df.apply(get_loop_id, axis=1)
    loops_bedpe = loop_df[['chr1', 's1', 'e1', 'chr2', 's2', 'e2', 'loop_id']]
    loop_pbt = pbt.BedTool.from_dataframe(loops_bedpe)

    # search for loops which overlap the gene body
    whole_gene_sgls = colocs_gene_pbt.pairtopair(loop_pbt, slop=gene_slop, type='both')
    wg_sgl_tmp = whole_gene_sgls.to_dataframe(disable_auto_names=True, header=None)
    wg_sgl_tmp.dropna(how='all', axis=1, inplace=True)
    wg_sgl_tmp['sgl_kind'] = 'gene'

    # search for loops which overlap the gene tss
    tss_sgls = colocs_tss_pbt.pairtopair(loop_pbt, slop=tss_slop, type='both')
    tss_sgl_tmp = tss_sgls.to_dataframe(disable_auto_names=True, header=None)
    tss_sgl_tmp.dropna(how='all', axis=1, inplace=True)
    tss_sgl_tmp['sgl_kind'] = 'tss'

    # merge both loop types 
    if wg_sgl_tmp.shape[0] > 0:
        if tss_sgl_tmp.shape[0] > 0:
            all_sgls = pd.concat([wg_sgl_tmp, tss_sgl_tmp], axis=0)
        else:
            all_sgls = wg_sgl_tmp
    else:
        if tss_sgl_tmp.shape[0] > 0:
            all_sgls = tss_sgl_tmp
        else:
            all_sgls = []

    
    if len(all_sgls) > 0: 
                
        all_sgls.columns = colocs_gene_bedpe.columns.tolist() + \
                            ['loop_chrA', 'loop_startA', 'loop_endA', \
                            'loop_chrB', 'loop_startB', 'loop_endB', \
                            'loop_id', 'sgl_kind']
        all_sgls = all_sgls.merge(colocs_df, on='coloc_id', suffixes=(None, '_remove'))
        all_sgls = all_sgls.merge(loop_df, on='loop_id', suffixes=(None, '_remove'))
        remove_cols = [x for x in all_sgls.columns.tolist() if '_remove' in x]
        all_sgls.drop(remove_cols, axis=1, inplace=True)

        # add loop source
        all_sgls['loop_source'] = loop_source

        ## Prioritize the tss loop 
        for grp, grp_df in all_sgls.groupby(['coloc_id', 'loop_id']):    

            if grp_df.shape[0] == 1:
                compiled_sgls.append(grp_df)
            else:        
                pr = grp_df.loc[grp_df.sgl_kind == 'tss']
                compiled_sgls.append(pr)
        

In [47]:
compiled_sgls = pd.concat(compiled_sgls)

In [48]:
compiled_sgls.shape

(23, 74)

## Extract SGLs between matched samples only

In [49]:
sgl_samplesheet = pd.read_table('config/sgl_samplesheets/sgl.samplesheet.tsv', comment='#')
sgl_samplesheet['master_sources'] = sgl_samplesheet['gwas_source'] + ':' \
                                    + sgl_samplesheet['eqtl_source'] + ':' \
                                    + sgl_samplesheet['ge_source'] + ':' \
                                    + sgl_samplesheet['loop_source']

In [50]:
compiled_sgls['master_sources'] = compiled_sgls['gwas_source'] + ':' \
                                    + compiled_sgls['eqtl_source'] + ':' \
                                    + compiled_sgls['ge_source'] + ':' \
                                    + compiled_sgls['loop_source']
compiled_sgls['eqtl_hichip_mapped'] = compiled_sgls['master_sources'].isin(sgl_samplesheet['master_sources'])
compiled_sgls = compiled_sgls.loc[compiled_sgls['eqtl_hichip_mapped'] == True]

## Add the full id

In [51]:
compiled_sgls['full_id'] = compiled_sgls['coloc_id'] + '.' \
                            + compiled_sgls['gwas_source'] + '.' \
                            + compiled_sgls['eqtl_source'] + '.' \
                            + compiled_sgls['ge_source'] + '.' \
                            + compiled_sgls['loop_source']

## Save  unique list of genes

In [52]:
compiled_sgls_genes = sorted(set(compiled_sgls.geneid.to_list()))
comp_fn = os.path.join(outdir, 'compiled_sgls_genes.txt')
with open(comp_fn, 'w') as f:
    for gene in compiled_sgls_genes:
        f.write(gene + '\n')
print(comp_fn)

results/main/GRCh37/sgls/notebook_output/Find_SGLs.Notebook_Managed.LD_SNPs_NOT_Included/compiled_sgls_genes.txt


## add colors

In [28]:
blue_hex = '#377eb8'.upper()
green_hex = '#4daf4a'.upper()
compiled_sgls['color'] = compiled_sgls['sgl_kind'].map({'tss': green_hex, 'gene': blue_hex})

## save to file

In [28]:
sgl_washu = compiled_sgls[['chr1', 's1', 'e1']]
sgl_washu['right'] = compiled_sgls['chr2'] + ':' \
                        + compiled_sgls['s2'].astype(str) + '-' \
                        + compiled_sgls['e2'].astype(str) + ','  \
                        + compiled_sgls['pp_H4_Coloc_Summary'].astype(str)
sgl_washu['full_id'] = compiled_sgls['full_id']
sgl_washu['color'] = compiled_sgls['color']
sgl_washu['chr_int'] = sgl_washu['chr1'].str.replace('chr', '').astype(int)
sgl_washu.sort_values(['chr_int', 's1', 'e1'], inplace=True)
sgl_washu.drop(['chr_int'], axis=1, inplace=True)

/mnt/BioHome/jreyna/tmp//ipykernel_20008/522069270.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sgl_washu['right'] = compiled_sgls['chr2'] + ':' \
/mnt/BioHome/jreyna/tmp//ipykernel_20008/522069270.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sgl_washu['full_id'] = compiled_sgls['full_id']
/mnt/BioHome/jreyna/tmp//ipykernel_20008/522069270.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

S

In [29]:
sgl_fn = os.path.join(outdir, 'sgl.track.refbed')

In [30]:
sgl_washu.iloc[:, 0:6].to_csv(sgl_fn, sep='\t', index=False, header=False)

In [31]:
cmd = 'bgzip -f {}\n'.format(sgl_fn)
cmd += 'tabix -p bed -f {}.gz'.format(sgl_fn)

In [32]:
print(cmd)

bgzip -f results/main/GRCh37/sgls/notebook_output/sgl.track.refbed
tabix -p bed -f results/main/GRCh37/sgls/notebook_output/sgl.track.refbed.gz


## View, Search and Query

In [33]:
compiled_sgls.genename

15       TMPRSS3
10       TMPRSS3
14       TMPRSS3
8        TMPRSS3
13       TMPRSS3
6        TMPRSS3
22    AL121787.1
23    AL121787.1
Name: genename, dtype: object

In [58]:
for rsid in compiled_sgls.rs_id.unique().tolist():
    print(rsid)

rs9981624
rs56297233
